In [ ]:
import optuna


study = optuna.create_study(
    study_name="my_first_study",
    direction="minimize",
    storage="sqlite:///example.db",
    load_if_exists=True,
)


def objective(trial):
    x = trial.suggest_float("x", -10, 10)
    return (x - 2) ** 2

study.optimize(objective, n_trials=30)

print("Baza danych stworzona ✅")


In [ ]:
from optuna.visualization import plot_optimization_history
plot_optimization_history(study)

In [4]:
import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances
from anomalib.models import Patchcore
from anomalib.data import MVTec
from anomalib.engine import Engine
path = "/mnt/c/Users/pawel/Explainable-hyperparameter-optimization-in-anomaly-detection/datasets/MVTec"
train_batch_size = 2
coreset_sampling_ratio = 0.1
limit_train_batches = 3

def objective(trial):
    params = {
        "backbone": trial.suggest_categorical("backbone", ["resnet18", "wide_resnet50_2"]),
        "coreset_sampling_ratio": trial.suggest_float("coreset_sampling_ratio", 0.01, 0.5),
        "num_neighbors": trial.suggest_int("num_neighbors", 1, 100),
    }


    model = Patchcore(**params)
    engine = Engine(max_epochs=1,
        accelerator="cpu",
        limit_train_batches=limit_train_batches
                    )


    datamodule = MVTec(root=path, category="bottle", train_batch_size=train_batch_size)


    engine.fit(datamodule=datamodule, model=model)
    metrics = engine.test(datamodule=datamodule, model=model)
    print(metrics)
    return metrics[0]["image_AUROC"]


study = optuna.create_study(
    direction="maximize",
    study_name="anomalib_patchcore_tuning",
    storage="sqlite:///anomalib3.db",
    load_if_exists=True,
)


study.optimize(objective, n_trials=10)


print("Najlepszy trial:")
trial = study.best_trial
print(f"  AUROC: {trial.value}")
print(f"  Parametry: {trial.params}")


plot_optimization_history(study).show()
plot_param_importances(study).show()

[I 2025-04-09 21:16:34,873] A new study created in RDB with name: anomalib_patchcore_tuning
INFO:anomalib.models.components.base.anomalib_module:Initializing Patchcore model.
INFO:timm.models._builder:Loading pretrained weights from Hugging Face hub (timm/wide_resnet50_2.racm_in1k)
INFO:timm.models._hub:[timm/wide_resnet50_2.racm_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
INFO:timm.models._builder:Missing keys (fc.weight, fc.bias) discovered while loading pretrained weights. This is expected if model is being adapted.
/tmp/ipykernel_13568/599406512.py:27: DeprecationWarning:

MVTec is deprecated and will be removed in a future version. Please use MVTecAD instead.

INFO:lightning_fabric.utilities.rank_zero:GPU available: False, used: False
INFO:lightning_fabric.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:lightning_fabric.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:anomalib.d

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:anomalib.models.image.patchcore.lightning_model:Aggregating the embedding extracted from the training set.
INFO:anomalib.models.image.patchcore.lightning_model:Applying core-set subsampling to get the embedding.


Selecting Coreset Indices.:   0%|          | 0/594 [00:00<?, ?it/s]

Selecting Coreset Indices.:  13%|█▎        | 76/594 [00:00<00:00, 758.77it/s]

Selecting Coreset Indices.:  26%|██▌       | 152/594 [00:00<00:00, 680.35it/s]

Selecting Coreset Indices.:  40%|████      | 238/594 [00:00<00:00, 753.66it/s]

Selecting Coreset Indices.:  53%|█████▎    | 315/594 [00:00<00:00, 701.84it/s]

Selecting Coreset Indices.:  65%|██████▍   | 386/594 [00:00<00:00, 578.99it/s]

Selecting Coreset Indices.:  75%|███████▌  | 447/594 [00:00<00:00, 575.38it/s]

Selecting Coreset Indices.: 100%|██████████| 594/594 [00:00<00:00, 652.33it/s]
INFO:lightning_fabric.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.
INFO:anomalib.callbacks.timer:Training took 17.03 seconds
INFO:li

Testing: |          | 0/? [00:00<?, ?it/s]

...

Testing: |          | 0/? [00:00<?, ?it/s]

INFO:anomalib.callbacks.timer:Testing took 31.544442892074585 seconds
Throughput (batch_size=32) : 2.6312083013789227 FPS


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        image_AUROC        │     0.995238184928894     │
│       image_F1Score       │    0.9841269850730896     │
│        pixel_AUROC        │    0.9807025194168091     │
│       pixel_F1Score       │    0.7271226048469543     │
└───────────────────────────┴───────────────────────────┘

[I 2025-04-09 21:54:07,187] Trial 9 finished with value: 0.995238184928894 and parameters: {'backbone': 'wide_resnet50_2', 'coreset_sampling_ratio': 0.060495426478212024, 'num_neighbors': 1}. Best is trial 0 with value: 1.0.


[{'image_AUROC': 0.995238184928894, 'image_F1Score': 0.9841269850730896, 'pixel_AUROC': 0.9807025194168091, 'pixel_F1Score': 0.7271226048469543}]
Najlepszy trial:
  AUROC: 1.0
  Parametry: {'backbone': 'wide_resnet50_2', 'coreset_sampling_ratio': 0.12644514780775395, 'num_neighbors': 60}
